## DATA

In [1]:
from _libs.pre_processing.pipeline import make_pipeline
from _libs.pre_processing.training_data import get_training_data, read_data
from _libs.paths import PATHS

In [2]:
X, y = read_data(PATHS["KIDNEY_DISEASE"], "class")

In [3]:
X.shape

(400, 24)

In [4]:
pipeline = make_pipeline(X)
pipeline.set_output(transform="pandas")

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['age', 'bp', 'bgr', 'bu',
                                                   'sc', 'sod', 'pot', 'hemo',
                                                   'pcv', 'wbcc', 'rbcc']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('encoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['sg', 'al', 'su', 'rbc',
                                                   'pc', 'pcc', 'ba', 'htn',
                                                   'dm', 'cad', 'appet', 'pe',
                                                   'ane'])]))])

In [5]:
X_train, X_test, y_train, y_test = get_training_data(X, y, pipeline)

## TPOT

In [6]:
from flaml import AutoML

automl = AutoML()
automl_settings = {
    "time_budget": 60,  # segundos
    "metric": 'accuracy',
    "task": 'classification',
    "log_file_name": "flaml.log",
    "verbose": 1,
    "n_splits": 10,
}

In [7]:
automl.fit(X_train=X_train, y_train=y_train, **automl_settings)

In [8]:
print("Best hyperparameters found: ", automl.best_config)
print("Best model: ", automl.model.estimator)
print("Best score: ", automl.metrics_for_best_config)

Best hyperparameters found:  {'n_estimators': 5, 'max_leaves': 5, 'min_child_weight': np.float64(0.14357258296800876), 'learning_rate': np.float64(0.16681157141301928), 'subsample': 1.0, 'colsample_bylevel': np.float64(0.8091595116713156), 'colsample_bytree': np.float64(0.9158265282522311), 'reg_alpha': np.float64(0.0017607866203119683), 'reg_lambda': np.float64(0.5212077054373687)}
Best model:  XGBClassifier(base_score=None, booster=None, callbacks=[],
              colsample_bylevel=np.float64(0.8091595116713156),
              colsample_bynode=None,
              colsample_bytree=np.float64(0.9158265282522311), device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=None,
              grow_policy='lossguide', importance_type=None,
              interaction_constraints=None,
              learning_rate=np.float64(0.16681157141301928), max_bin=None,
              max_cat_threshold=None, max_cat_to_oneh

In [9]:
from sklearn.metrics import classification_report

y_pred = automl.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         ckd       0.99      0.99      0.99       205
      notckd       0.97      0.98      0.98       115

    accuracy                           0.98       320
   macro avg       0.98      0.98      0.98       320
weighted avg       0.98      0.98      0.98       320

